## 5.11 RNN-LSTM-例子1-分类

**【知识框架】**：
- 第一步：导入数据
- 第二步：参数设置
    1. 学习率
    2. 训练次数上限
    3. 随机选取训练的一小部分图片数（batch_size）
    4. 每个图像的行数（image shape：28 * 28，其中行数28，每行的像素28）
    5. 每行像素：28
    6. 隐藏层输出：128
    7. 分类：10
- 第三步：定义x,y 的 placeholder 和 weights, biases 的初始状况.
- 第四步：**定义RNN的主体框架**
- 第五步：训练RNN

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


- 第一步：导入数据

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


- 第二步：参数设置

In [4]:
lr = 0.001 #学习率

training_iters = 100000 # train step 上限
batch_size = 128

n_inputs = 28 # MNIST data input (img shape: 28*28)
n_steps = 28  # time steps

n_hidden_units = 128  # neurons in hidden layer
n_classes = 10 # MNIST classes (0-9 digits)

- 第三步：定义x,y 的 placeholder 和 weights, biases 的初始状况.

In [5]:
# x y placeholder
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# 对 weights biases 初始值的定义
weights = {
#     shape(28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
#     shape(128,10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}

biases = {
#     shape(128,)
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units,])),
#     shape(10,)
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes,]))
}

- 第四步：**定义RNN的主体框架**

这个 RNN 总共有 3 个组成部分 ( input_layer, cell, output_layer). 首先我们先定义 input_layer:

- 4.1 input_layer

In [6]:
def RNN(X, weights, biases):
    #原始的 X 是3维数据, 我们需要把它变成2维数据才能使用weights的矩阵乘法
    #X = = > (128 batches * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])
    
    # X_in = W*X + b
    X_in = tf.matmul(X, weights['in']) + biases['in']
     # X_in ==> (128 batches, 28 steps, 128 hidden) 换回3维
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    
    # 使用 basic LSTM Cell.
    cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias = 1.0,
                                     state_is_tuple = True)
    # 初始化全零state
    init_state = cell.zero_state(batch_size, dtype = tf.float32) 
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, 
                                             initial_state=init_state,
                                            time_major=False)
    # 把 outputs 变成 列表 [(batch, outputs)..] * steps
#     将（x,y,z）转成（y,x,z）即（batch, step, inputs）= = > (step, batch, inputs)
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']
    return results
    

- 4.2 cell

(1). 使用 tf.nn.dynamic_rnn(cell, inputs) (推荐). 
(2). state_is_tuple=True, 对于 lstm 来说, state可被分为元组(c_state, h_state),即主线state（c_state）和分线state(h_state)

In [7]:
#     # 使用 basic LSTM Cell.
#     lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias = 1.0,
#                                      state_is_tuple = True)
#     # 初始化全零state
#     init_state = lstm_cell.zero_state(batch_size, dtype = tf.float32)

如果使用`tf.nn.dynamic_rnn(cell, inputs)`, 我们要确定 `inputs `的格式. `tf.nn.dynamic_rnn` 中的 time_major 参数会针对不同 inputs 格式有不同的值.

1. 如果 `inputs` 为 (batches, steps, inputs) ==> `time_major=False`;
2. 如果 `inputs` 为 (steps, batches, inputs) ==> `time_major=True;`

In [8]:
# outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state,
#                                          time_major = False)

- 4.3 output_layer

**方式一**: 直接调用final_state 中的 h_state (final_state[1]) 来进行运算

In [9]:
# result = tf.matmul(final_state[1], weights['out']) + biases['out']

**方式二：** 调用最后一个 outputs (在这个例子中,和上面的final_state[1]是一样的)

In [10]:
# 把 outputs 变成 列表 [(batch, outputs)..] * steps
# outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
# result = tf.matmul(outputs[-1], weights['out']) + biases['out']

- 4.4 定义cost和train_op

In [11]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

- 第五步：训练RNN

In [13]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
                x: batch_xs,
                y: batch_ys,
            }))
        step += 1

0.1015625
0.609375
0.734375
0.7578125
0.8671875
0.828125
0.890625
0.921875
0.828125
0.875
0.8984375
0.9375
0.9453125
0.90625
0.8984375
0.9296875
0.9375
0.953125
0.9375
0.9453125
0.953125
0.953125
0.96875
0.9609375
0.9296875
0.9921875
0.9765625
0.96875
0.96875
0.96875
0.984375
0.9609375
0.9765625
0.953125
0.96875
0.9609375
0.9765625
0.9453125
0.9609375
0.984375
